In [1]:
# Step 1: Import Libraries and Load the Model
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model

In [3]:

# Load the IMDB dataset word index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

In [4]:
reverse_word_index

{34701: 'fawn',
 52006: 'tsukino',
 52007: 'nunnery',
 16816: 'sonja',
 63951: 'vani',
 1408: 'woods',
 16115: 'spiders',
 2345: 'hanging',
 2289: 'woody',
 52008: 'trawling',
 52009: "hold's",
 11307: 'comically',
 40830: 'localized',
 30568: 'disobeying',
 52010: "'royale",
 40831: "harpo's",
 52011: 'canet',
 19313: 'aileen',
 52012: 'acurately',
 52013: "diplomat's",
 25242: 'rickman',
 6746: 'arranged',
 52014: 'rumbustious',
 52015: 'familiarness',
 52016: "spider'",
 68804: 'hahahah',
 52017: "wood'",
 40833: 'transvestism',
 34702: "hangin'",
 2338: 'bringing',
 40834: 'seamier',
 34703: 'wooded',
 52018: 'bravora',
 16817: 'grueling',
 1636: 'wooden',
 16818: 'wednesday',
 52019: "'prix",
 34704: 'altagracia',
 52020: 'circuitry',
 11585: 'crotch',
 57766: 'busybody',
 52021: "tart'n'tangy",
 14129: 'burgade',
 52023: 'thrace',
 11038: "tom's",
 52025: 'snuggles',
 29114: 'francesco',
 52027: 'complainers',
 52125: 'templarios',
 40835: '272',
 52028: '273',
 52130: 'zaniacs',

In [5]:
model = load_model('Simple_RNN_imdb.h5')

In [6]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

### embedding layer
In the Embedding layer, the number of parameters is calculated as:

number of unique words (input_dim) × embedding dimension (output_dim)

In your case:

input_dim (vocabulary size) = 10,000 (max_features)
output_dim (embedding dimension) = 128
So, the total parameters = 10,000 × 128 = 1,280,000

Each word in the vocabulary gets its own 128-dimensional vector, and all these vectors are trainable parameters in the Embedding

### simple RNN layer


For a SimpleRNN layer with units = N and input size M (from the Embedding layer):

Each RNN unit needs:
Input weights: One weight for each input feature (M per unit)
Recurrent weights: One weight for each unit’s previous output (N per unit)
Bias: One bias per unit
So, for each unit, the total parameters = M (input) + N (recurrent) + 1 (bias)

But you have N units, so you multiply the whole sum by N:

Total parameters = N × (M + N + 1)

The bias is not multiplied by M, it’s just one per unit.
The formula is not (M × N) + N (bias), but N × (M + N + 1).
Example with your numbers:

M = 128 (embedding output)
N = 128 (RNN units)
So:
128 × (128 + 128 + 1) = 128 × 257 = 32,896

This is how Keras and most deep learning libraries count RNN parameters. Each unit has its own set of input weights, recurrent weights, and a bias.


### Dense Layer:

If the Dense layer has 1 unit (for binary classification) and input from SimpleRNN is 128:
Formula: input_dim * units + units
For your case: 128 * 1 + 1 = 129
This includes weights and a bias.
So, the parameters in these layers are the trainable weights and biases that the model learns during training.

### optimizer params

The "Optimizer params: 2" refers to the number of internal variables (not model weights) that the optimizer itself maintains during training.

For the default "adam" optimizer in Keras, it keeps track of two variables for each model weight:

The first moment (mean of gradients, called "m")
The second moment (uncentered variance of gradients, called "v")
However, in some Keras/TensorFlow versions, the summary may only show the number of global optimizer variables, not per-weight. In your case, "Optimizer params: 2" means the optimizer is maintaining 2 global variables (such as iteration count and learning rate), not per-weight statistics.

This number is not related to the model's trainable parameters, but to the optimizer's own state. It is normal to see "Optimizer params: 2" for simple optimizers or certain Keras versions.

###
In Keras, model.get_weights() returns a list of all the trainable parameters (weights and biases) of your model. These include:

The embedding matrix from the Embedding layer (word vectors)
The input weights, recurrent weights, and biases from the SimpleRNN layer
The weights and bias from the Dense (output) layer
These weights are the values that the model learns during training to minimize the loss and make accurate predictions.

In [7]:
model.get_weights()


[array([[-0.08159558, -0.13740067,  0.11182784, ..., -0.13934872,
         -0.13982996, -0.11285004],
        [-0.01432273, -0.02302826, -0.02890527, ...,  0.04261259,
         -0.0162134 , -0.02636589],
        [-0.03054017,  0.00146166, -0.01525505, ...,  0.05489467,
         -0.07301357, -0.0840021 ],
        ...,
        [-0.00952106,  0.02877504, -0.05138839, ..., -0.02254986,
          0.07089985, -0.01159531],
        [ 0.09541053, -0.06159914, -0.04211431, ..., -0.03389214,
          0.04909036, -0.04809663],
        [-0.09906552,  0.06551702,  0.06295351, ...,  0.15320624,
         -0.09625072,  0.19128565]], shape=(10000, 128), dtype=float32),
 array([[-6.9146745e-02, -9.6393451e-02, -1.4084516e-02, ...,
         -1.4028472e-02,  7.3753908e-02, -7.2496863e-05],
        [-5.4915235e-03, -6.3313499e-02,  9.6932285e-02, ...,
          5.9363090e-02, -5.2407585e-02, -7.0131496e-02],
        [ 1.9900871e-03,  1.5087021e-02, -1.0056186e-01, ...,
          7.2615348e-02, -9.4710283e

In [58]:
# Step 2: Helper Functions
# Function to decode reviews
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

# Function to preprocess user input
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    #print((encoded_review))
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    print(padded_review)
    return padded_review

In [9]:
### Prediction  function

def predict_sentiment(review):
    preprocessed_input=preprocess_text(review)

    prediction=model.predict(preprocessed_input)

    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    
    return sentiment, prediction[0][0]



In [10]:
# Step 4: User Input and Prediction
# Example review for prediction
example_review = "This movie was fantastic! The acting was great and the plot was thrilling."

sentiment,score=predict_sentiment(example_review)

print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
Review: This movie was fantastic! The acting was great and the plot was thrilling.
Sentiment: Positive
Prediction Score: 0.9381566643714905


In [11]:
example_review = "This movie was fantastic! The acting was great and the plot was thrilling."
print(preprocess_text(example_review))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0 

In [13]:
a=[14, 20, 16, 5, 4, 116, 16, 87, 5, 4, 114, 16, 5]
print(decode_review(a))

this movie was and the acting was great and the plot was and


In [60]:
example_review = "You are a stupid boy, and i have never seen that type of stupidity in someone, its very pathetic to live with such kind of people like you, hope you will die one day soon ."

sentiment,score=predict_sentiment(example_review)

print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   

In [30]:
reverse_word_index[50001]

'botanist'